In [ ]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device
# Create robot arm object
Arm = Arm_Device()
time.sleep(.1)
for i in range(3):
    Arm.Arm_RGB_set(255, 255, 255)
    time.sleep(.1)
    Arm.Arm_RGB_set(0,0, 0)
    time.sleep(.1)
time.sleep(1)

In [ ]:
Arm.Arm_serial_servo_write6(90, 90, 90, 90, 90, 90, 2000)

In [ ]:
Arm.Arm_serial_servo_write6(90, 130, 0, 0, 90, 90, 2000)

In [ ]:
import numpy as np
import cv2

def get_colors(img):
    color_map = {
        "red": [],
        "yellow": [],
        "green": [],
        "blue": [],
    }
    
    thresholds = {
        "red": {
            "lower": [0, 43, 200],
            "upper": [70, 80, 255]
        },
        "yellow": {
            "lower": [80, 220, 220],
            "upper": [150, 255, 255]
        },
        "green": {
            "lower": [15, 100, 46],
            "upper": [70, 150, 90]
        },
        "blue": {
            "lower": [130, 80, 20],
            "upper": [224, 120, 90]
        },
    }
    
    BGR = {}
    S = {}
    color_name={}
    img = cv2.resize(img, (160, 120), )
    # Convert color image to HSV
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Take out the H, S, and V values of each row and column in turn and put them into the container.
    for i in range(0, 160):
        for j in range(0, 120):
            BGR[(i, j)] = [c for c in img[j, i]]
            S[(i, j)] = HSV[j, i][1]
            
    precision = 4
#     threshold = 0.9 * max(S.values())
    
    for point, color in BGR.items():
        
        if all(color[i] >= thresholds["red"]["lower"][i] for i in range(3)) and all(color[i] <= thresholds["red"]["upper"][i] for i in range(3)):
            color_map["red"].append(point)
        elif all(color[i] >= thresholds["yellow"]["lower"][i] for i in range(3)) and all(color[i] <= thresholds["yellow"]["upper"][i] for i in range(3)):
            color_map["yellow"].append(point)
        elif all(color[i] >= thresholds["green"]["lower"][i] for i in range(3)) and all(color[i] <= thresholds["green"]["upper"][i] for i in range(3)):
            color_map["green"].append(point)
        elif all(color[i] >= thresholds["blue"]["lower"][i] for i in range(3)) and all(color[i] <= thresholds["blue"]["upper"][i] for i in range(3)):
            color_map["blue"].append(point)
#     print(S)


    for color in ["red", "yellow", "green", "blue"]:
        points = [x for x in color_map[color]]
        if not points:
            color_map[color] = (-1, -1)
            continue
        
        saturation = [S[p] for p in points]
#         threshold = 0.9 * max(saturation)
        
#         points = [p for p in points if S[p] > threshold]
#         saturation = [S[p] for p in points]
        
#         avg_x = np.average([p[0] for p in points], weights = saturation)
#         avg_y = np.average([p[1] for p in points], weights = saturation)
        avg_x = np.mean([p[1] for p in points])
        avg_y = np.mean([p[0] for p in points])
        color_map[color] = (avg_y, avg_x)
    
    return color_map, BGR

In [ ]:
import cv2
import ipywidgets.widgets as widgets
import threading
image_widget = widgets.Image(format='jpeg', width=600, height=500) #Set up the camera display component
color_widget = widgets.Image(format='jpeg', width=600, height=500)
image_container = widgets.HBox([image_widget, color_widget])
display(image_container) #Show camera components

#bgr8 to jpeg format
#image is 640x480
import enum
import cv2
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def monitor():
    image = cv2.VideoCapture(1) #open camera
    if not image.isOpened():
        print("couldn't open the camera")
        return
    print("running")
    Arm.Arm_RGB_set(0, 255, 0)
    point_colors = {
        "red": (0,0,255),
        "yellow": (0,255,255),
        "green": (0,255,0),
        "blue": (255,0,0)
        }
    
    while True:
        ret, frame = image.read()
        color_image = np.ones((500, 600, 3), dtype=np.uint8) * 255
        try:
            data = open("colordata.csv", mode="w")
            color_map, BGR = get_colors(frame)
            for color, point in color_map.items():
                cv2.circle(frame, ( int(round(point[0] * 4)), int(round(point[1] * 4 )) ), 10, point_colors[color], -1)
                print(color, ( int(round(point[0] * 4)), int(round(point[1] * 4 )) ))

            for point, clr in BGR.items():
                center = tuple([4*x for x in point])
                color = tuple([int(value) for value in clr])
                cv2.circle(color_image, center, 2, color, -1)
                data.write(f"{center},{color}\n")
            
#             cv2.circle(frame, (200, 300), 3, (255, 255, 255), -1)
            image_widget.value = bgr8_to_jpeg(frame)
            color_widget.value = bgr8_to_jpeg(color_image)
            Arm.Arm_RGB_set(0, 255, 0)
            time.sleep(1)
            data.close()
            return
        except Exception as e:
            Arm.Arm_RGB_set(255, 0, 0)
            print(e)
            return
            
        time.sleep(1)

In [ ]:
camera_thread = threading.Thread(target=monitor)
camera_thread.start()

In [ ]:

# Camera matrix (intrinsics)
K = np.array([[fx, 0, cx],  # fx and fy are focal lengths, cx and cy are the principal point
              [0, fy, cy],
              [0, 0, 1]])

# 2D image point (pixel coordinates)
u, v = 100, 150

# Assuming some depth value (Z) in camera space, for example Z = 5 meters
Z = 5

p_image = np.array([u, v, 1])
K_inv = np.linalg.inv(K)
p_camera_normalized = K_inv @ p_image

# scale by depth to get the 3D coordinates in camera space
p_camera = p_camera_normalized * Z

print("3D Point in Camera Coordinates:", p_camera)
